## 1. PP-LCNetv2 Introduction
PP-LCNetv2 series model is a lightweight convolution neural network model proposed by PaddleCV team. This series model is improved based on PP-LCNetv1 and focuses more on higher model performance. The accuracy of this model can reach 77.04% on the ImageNet1k dataset. Meanwhile, the inference latency is about 4.32ms. In addition, considering the extensive use of OpenVINO inference framework in the industry, PP-LCNetv2 series model is specially optimized for Intel CPU combined with OpenVINO.

The PP-LCNetv2 series model was developed and trained based on PaddleClas. For more information on PP-LCNet series models, see [PaddleClas-PPLCNetv2](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.5/docs/en/models/PP-LCNetV2_en.md).

## 2. Performance

The evaluation results of PP-LCNetV2 series models on ImageNet1k dataset are shown as follows:

| Model | Params(M) | FLOPs(M) | Top-1 Acc(\%) | Top-5 Acc(\%) | Latency(ms) |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| <b>PPLCNetv2_base<b>  | <b>6.6<b> | <b>604<b>  | <b>77.04<b> | <b>93.27<b> | <b>4.32<b> |
| <b>PPLCNetv2_base_ssld<b>  | <b>6.6<b> | <b>604<b>  | <b>80.07<b> | <b>94.87<b> | <b>4.32<b> |

`_ssld` represents the model after using `SSLD distillation`. For details about `SSLD distillation`, see [SSLD distillation](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.5/docs/en/advanced_tutorials/knowledge_distillation_en.md).

Compared with other lightweight networks:

| Model | Params(M) | FLOPs(M) | Top-1 Acc(\%) | Top-5 Acc(\%) | Latency(ms) |
|:--:|:--:|:--:|:--:|:--:|:--:|
| MobileNetV3_Large_x1_25 | 7.4 | 714  | 76.4 | 93.00 | 5.19 |
| PPLCNetv1_x2_5  | 9 | 906  | 76.60 | 93.00 | 7.25 |
| <b>PPLCNetv2_base<b>  | <b>6.6<b> | <b>604<b>  | <b>77.04<b> | <b>93.27<b> | <b>4.32<b> |
| <b>PPLCNetv2_base_ssld<b>  | <b>6.6<b> | <b>604<b>  | <b>80.07<b> | <b>94.87<b> | <b>4.32<b> |

## 3. Quick Start

### 3.1 Inference:
* Install the relevant Python packages

(Remove the"!" when not running on the Jupyter Notebook)


Run the following command to install if CUDA9、CUDA10 or CUDA11 is available.

In [ ]:
!pip install paddlepaddle-gpu

Run the following command to install if GPU device is unavailable.

In [ ]:
!pip install paddlepaddle

Install paddleclas

In [ ]:
!pip install paddleclas

* Quick Start

Congratulations! Now that you've successfully installed PaddleClas, you can experience the image classification effects.

In [ ]:
!wget https://gitee.com/paddlepaddle/PaddleClas/raw/release/2.5/docs/images/inference_deployment/whl_demo.jpg
!paddleclas --model_name=PPLCNetv2_base  --infer_imgs="./whl_demo.jpg"

The above command results are as follows:

class_ids: [8, 7, 86, 82, 83], scores: [0.8859, 0.07156, 0.00588, 0.00047, 0.00034], label_names: ['hen', 'cock', 'partridge', 'ruffed grouse, partridge, Bonasa umbellus', 'prairie chicken, prairie grouse, prairie fowl'], filename: docs/images/inference_deployment/whl_demo.jpg
Predict complete

### 3.2 Training
* PP-LCNetV2 series models are implemented based on PaddleClas. For details of model training, please refer to[Training, Evaluation and Inference](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.5/docs/en/models/PP-LCNetV2_en.md).

## 4. Algorithm

### 4.1 Overall Structure

The overall structure of the network is shown in the figure below:

<div align="center">
<img src="https://user-images.githubusercontent.com/12560511/200776415-19f7f09d-f8e9-4b64-bbcd-72ac418c23cf.png"  width = "80%"  />
</div>

### 4.2 Model Details

PP-LCNetV2 model is optimized on the basis of PP-LCNetV1. It mainly uses the reparameterization strategy to combine the depthwise convolutions with different kernel sizes, and optimize the point convolution and Shortcut, etc.

#### 4.2.1  Re-parameterization

The size of the convolution kernel affects the size of the model's receptive field, which affects the model's ability to capture more global or local features. In order to help the model build different scales features, we use 5*5, 3*3 and 1*1 size convolution kernel. The details is shown in the figure below.

<div align="center">
<img src="https://user-images.githubusercontent.com/12560511/201664156-be8f5dc6-5345-436a-962e-079139f378d5.png"  width = "80%"  />
</div>

#### 4.2.2 PW Conv

The depthwise separable convolution usually consists of a layer of DW convolution and a layer of PW convolution to replace the standard convolution. In order to make the depth separable convolution have a stronger fitting ability, we try to use two layers of PW convolution. Meanwhile, in order to control the efficiency of the model, the two-layer PW convolution is set as: The first squeeze the feature map on the channel dimension, and the second expand to restore the feature map channel, as shown in the figure below. Experiments show that this strategy can significantly improve model performance. Meanwhile, in order to balance the impact on model efficiency, PPLCNetV2 only uses this strategy in Stage4.

<div align="center">
<img src="https://user-images.githubusercontent.com/12560511/201664376-e3cdd4ac-0767-4773-a6cc-b6885796bc1e.png"  width = "80%"  />
</div>

#### 4.2.3 Shortcut

The residual structure has been widely used by many models since it was proposed, but in lightweight convolutional neural networks, the elementwise addition operation brought by the residual structure will affect the speed of the model. We experimented on the influence of shortcut on the model at different stage. Finally, we only used Shortcut in the last block, as shown in the figure below.

<div align="center">
<img src="https://user-images.githubusercontent.com/12560511/201664472-42139a13-28a1-43b9-92d7-d355f10ed56d.png"  width = "80%"  />
</div>

#### 4.2.4 Activation Function

In the current lightweight convolutional neural networks, ReLU and Hard-Swish activation functions are most commonly used. Although Hard-Swish is usually better in terms of model performance, we found that some inference platforms have unsatisfactory efficiency optimization for Hard-Swish activation functions. Therefore, in order to give consideration to universality, PP-LCNetV2 uses the ReLU activation function by default, and our tests show that the ReLU activation function has little impact on the performance of larger models.


#### 4.2.5 SE

Although SE module can significantly improve model performance, its impact on model speed should not be ignored. In PP-LCNetV1, we found that the use of SE module in the back of the model can obtain the maximum benefit. In the optimization process of PP-LCNetV2, we did further experiments on the position of SE module by Stage, and found that better balance could be achieved by using it in Stage4.

## 5. Notes
The inference speed of PP-LCNet series model were tested at Intel Xeon Gold 6271C CPU with PaddlePaddle inference framework, the batch size was 1 and the number of threads was 10.

## 6. Relevant Papers and Citations